# Credit Risk Problem

In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.3f}'.format)

## 1. Dataset import

In [3]:
DATA_DIR = 'data'
DATA_FILE_NAME = 'credit.csv'

In [4]:
data_file_path = os.path.join(DATA_DIR, DATA_FILE_NAME)

In [5]:
df = pd.read_csv(data_file_path).set_index('Unnamed: 0')

In [6]:
df.index.name = 'index'

In [7]:
df.index.is_unique

True

In [8]:
df.shape

(1000, 22)

In [9]:
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,residence_history,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job,gender
index,,,,,,,,,,,,,,,,,,,,,,
0,-43.000,6,critical,radio/tv,1169,nan,13 years,4,single,none,6 years,real estate,67,none,own,2,0,1,2349339647.000,yes,skilled employee,male
1,75.000,48,repaid,radio/tv,5951,89.000,2 years,2,NaN,none,5 months,real estate,22,none,own,1,1,1,nan,yes,skilled employee,female
2,nan,12,critical,education,2096,24.000,5 years,2,single,none,4 years,real estate,49,none,own,1,0,2,nan,yes,unskilled resident,male
3,-32.000,42,repaid,furniture,7882,9.000,5 years,2,single,guarantor,13 years,building society savings,45,none,for free,1,0,2,nan,yes,skilled employee,male
4,-23.000,24,delayed,car (new),4870,43.000,3 years,3,single,none,13 years,unknown/none,53,none,for free,2,1,2,nan,yes,skilled employee,male


In [10]:
TARGET = 'default'

## 2. Dataset overview
### 2.1. General overview

The target is the variable **default**.

The data has the following structure:
- ``Observation_id``: unique observation id
- ``Checking_balance``: Status of existing checking account. (German currency)
- ``Savings_balance``: Savings account/bonds (German currency)
- ``Installment_rate``: Installment rate in percentage of disposable income
- ``Personal_status``: Personal status and sex
- ``Residence_history``: Present residence since
- ``Installment_plan``: Other installment plans
- ``Existing_credits``: Number of existing credits at this bank
- ``Dependents``: Number of people being liable to provide maintenance for
- ``Default``: 1 is a good loan, 2 is a defaulting one.

In [11]:
df[TARGET].value_counts()

0    700
1    300
Name: default, dtype: int64

In [12]:
df[TARGET].mean()

0.3

In [13]:
overview_df = pd.DataFrame(index=df.columns)

In [14]:
overview_df['dtypes'] = df.dtypes

In [15]:
overview_df['isna'] = df.isna().sum()

In [16]:
overview_df

,dtypes,isna
checking_balance,float64,394
months_loan_duration,int64,0
credit_history,object,0
purpose,object,0
amount,int64,0
savings_balance,float64,183
employment_length,object,62
installment_rate,int64,0
personal_status,object,310
other_debtors,object,0


### 2.2. Variables types

In [17]:
df['credit_history'].value_counts(dropna=False)

repaid                    530
critical                  293
delayed                    88
fully repaid this bank     49
fully repaid               40
Name: credit_history, dtype: int64

In [18]:
df['purpose'].value_counts(dropna=False).head()

radio/tv      280
car (new)     234
furniture     181
car (used)    103
business       97
Name: purpose, dtype: int64

In [19]:
df['employment_length'].value_counts(dropna=False).head(10)

1 years     122
2 years     110
3 years     107
NaN          62
7 years      57
5 years      46
6 years      45
4 years      44
13 years     26
19 years     24
Name: employment_length, dtype: int64

In [20]:
df['personal_status'].value_counts(dropna=False).head()

single      548
NaN         310
married      92
divorced     50
Name: personal_status, dtype: int64

In [21]:
df['other_debtors'].value_counts(dropna=False).head()

none            907
guarantor        52
co-applicant     41
Name: other_debtors, dtype: int64

In [22]:
df['residence_history'].value_counts(dropna=False).head(10)

NaN         130
2 years      43
8 months     38
1 years      37
3 years      36
0 months     34
4 years      33
9 months     29
1 months     28
10 years     27
Name: residence_history, dtype: int64

In [23]:
df['property'].value_counts(dropna=False).head()

other                       332
real estate                 282
building society savings    232
unknown/none                154
Name: property, dtype: int64

In [24]:
df['installment_plan'].value_counts(dropna=False).head()

none      814
bank      139
stores     47
Name: installment_plan, dtype: int64

In [25]:
df['housing'].value_counts(dropna=False).head()

own         713
rent        179
for free    108
Name: housing, dtype: int64

In [26]:
df['telephone'].dropna().head()

index
0    2349339647.000
5    2345787903.000
7    2349961683.000
12   2343498747.000
18   2340724318.000
Name: telephone, dtype: float64

In [27]:
df['telephone'].isna().mean()

0.596

In [28]:
df['telephone_present'] = df['telephone'].notna().astype(int)

In [29]:
df['telephone_present'].value_counts()

0    596
1    404
Name: telephone_present, dtype: int64

In [30]:
df['foreign_worker'].value_counts(dropna=False).head()

yes    963
no      37
Name: foreign_worker, dtype: int64

In [31]:
df['job'].value_counts(dropna=False).head()

skilled employee           630
unskilled resident         200
mangement self-employed    148
unemployed non-resident     22
Name: job, dtype: int64

In [32]:
df['gender'].value_counts(dropna=False).head()

male      690
female    310
Name: gender, dtype: int64

In [33]:
CONTINUOUS_VARIABLES = [
    'checking_balance',
    'months_loan_duration',
    'amount',
    'savings_balance',
    'installment_rate',
    'age',
    'existing_credits',
    'dependents',
]

NOMINAL_VARIABLES = [
    'credit_history',
    'purpose',
    'personal_status',
    'other_debtors',
    'property',
    'installment_plan',
    'housing',
    'job'
]

BINARY_VARIABLES = [
    'telephone_present',
    'foreign_worker',
    'gender',
]

ORDINAL_VARIABLES = [
    'employment_length',
    'residence_history',
]

### 2.3. Outliers inspection

In [34]:
def out_of_iqr(s):
    Q1, Q3 = s.quantile([.25, .75])
    IQR = Q3 - Q1
    
    return (s < Q1 - 1.5 * IQR) | (s > Q3 + 1.5 * IQR)

In [35]:
pd.Series({c: out_of_iqr(df[c]).sum() for c in CONTINUOUS_VARIABLES})

checking_balance         55
months_loan_duration     70
amount                   72
savings_balance         173
installment_rate          0
age                      23
existing_credits          6
dependents              155
dtype: int64

## 3. Preprocessing
### 3.1. Nominal variables
#### Dummy variables creation

In [36]:
dummies_df = pd.get_dummies(df[NOMINAL_VARIABLES + BINARY_VARIABLES], dummy_na=True, drop_first=True)

In [37]:
dummies_df.shape

(1000, 40)

In [38]:
df.drop(labels=NOMINAL_VARIABLES + BINARY_VARIABLES, axis=1, inplace=True)

In [39]:
df = df.join(dummies_df)

In [40]:
df.head()

,checking_balance,months_loan_duration,amount,savings_balance,employment_length,installment_rate,residence_history,age,existing_credits,default,dependents,telephone,telephone_present,credit_history_delayed,credit_history_fully repaid,credit_history_fully repaid this bank,credit_history_repaid,credit_history_nan,purpose_car (new),purpose_car (used),purpose_domestic appliances,purpose_education,purpose_furniture,purpose_others,purpose_radio/tv,purpose_repairs,purpose_retraining,purpose_nan,personal_status_married,personal_status_single,personal_status_nan,other_debtors_guarantor,other_debtors_none,other_debtors_nan,property_other,property_real estate,property_unknown/none,property_nan,installment_plan_none,installment_plan_stores,installment_plan_nan,housing_own,housing_rent,housing_nan,job_skilled employee,job_unemployed non-resident,job_unskilled resident,job_nan,foreign_worker_yes,foreign_worker_nan,gender_male,gender_nan
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-43.000,6,1169,nan,13 years,4,6 years,67,2,0,1,2349339647.000,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0
1,75.000,48,5951,89.000,2 years,2,5 months,22,1,1,1,nan,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0
2,nan,12,2096,24.000,5 years,2,4 years,49,1,0,2,nan,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0
3,-32.000,42,7882,9.000,5 years,2,13 years,45,1,0,2,nan,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0
4,-23.000,24,4870,43.000,3 years,3,13 years,53,2,1,2,nan,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0


### 3.2. Ordinal variables
#### Transformation to continuos

In [41]:
ORDINAL_VARIABLES

['employment_length', 'residence_history']

In [42]:
df['employment_length'].nunique()

31

In [43]:
df['employment_length'].unique()

array(['13 years', '2 years', '5 years', '3 years', '11 years', '4 years',
       nan, '6 months', '5 months', '16 years', '1 years', '17 years',
       '3 months', '9 years', '4 months', '10 years', '10 months',
       '1 months', '7 months', '19 years', '7 years', '14 years',
       '18 years', '0 months', '15 years', '9 months', '6 years',
       '8 years', '12 years', '11 months', '2 months', '8 months'],
      dtype=object)

In [44]:
df['employment_length'] = df['employment_length'].astype(str)

In [45]:
def term_to_months(x):
    if x == 'nan':
        return np.nan
    
    term, unit = x.split(' ')
    
    if unit == 'months':
        return int(term)
    
    return 12 * int(term)

In [46]:
df['employment_length_m'] = df['employment_length'].apply(term_to_months)

In [47]:
df[['employment_length', 'employment_length_m']].head()

,employment_length,employment_length_m
index,,
0,13 years,156.000
1,2 years,24.000
2,5 years,60.000
3,5 years,60.000
4,3 years,36.000


In [48]:
df['residence_history'].nunique()

36

In [49]:
df['residence_history'].unique()

array(['6 years', '5 months', '4 years', '13 years', '8 years',
       '12 years', '3 months', '24 years', '10 months', nan, '10 years',
       '19 years', '0 months', '7 years', '3 years', '8 months',
       '7 months', '14 years', '1 years', '16 years', '6 months',
       '2 years', '20 years', '11 months', '21 years', '5 years',
       '9 months', '2 months', '15 years', '11 years', '18 years',
       '22 years', '23 years', '1 months', '9 years', '4 months',
       '17 years'], dtype=object)

In [50]:
df['residence_history'] = df['residence_history'].astype(str)

In [51]:
df['residence_history_m'] = df['residence_history'].apply(term_to_months)

In [52]:
df[['residence_history', 'residence_history_m']].head()

,residence_history,residence_history_m
index,,
0,6 years,72.000
1,5 months,5.000
2,4 years,48.000
3,13 years,156.000
4,13 years,156.000


### 3.3. Continuous variables
#### Outliers cut

In [53]:
df['dependents'].value_counts()

1    845
2    155
Name: dependents, dtype: int64

In [54]:
CONTINUOUS_VARIABLES = CONTINUOUS_VARIABLES + ['employment_length_m', 'residence_history_m']

In [55]:
CONTINUOUS_VARIABLES

['checking_balance',
 'months_loan_duration',
 'amount',
 'savings_balance',
 'installment_rate',
 'age',
 'existing_credits',
 'dependents',
 'employment_length_m',
 'residence_history_m']

In [56]:
def cut_by_iqr(s):
    Q1, Q3 = s.quantile([.25, .75])
    IQR = Q3 - Q1
    
    MIN = Q1 - 1.5 * IQR
    MAX = Q3 + 1.5 * IQR
    
    return np.where(s < MIN, MIN, np.where(s > MAX, MAX, s))

In [57]:
df['dependents'].value_counts()

1    845
2    155
Name: dependents, dtype: int64

In [58]:
CONTIONUOUS_EXCLUSIONS = ['dependents']

In [59]:
for c in CONTINUOUS_VARIABLES:
    
    if c in CONTIONUOUS_EXCLUSIONS:
        continue
    
    df[c] = cut_by_iqr(df[c])

In [60]:
pd.Series({c: out_of_iqr(df[c]).sum() for c in CONTINUOUS_VARIABLES})

checking_balance          0
months_loan_duration      0
amount                    0
savings_balance           0
installment_rate          0
age                       0
existing_credits          0
dependents              155
employment_length_m       0
residence_history_m       0
dtype: int64

#### Nulls replacement

In [61]:
df[CONTINUOUS_VARIABLES].isna().sum()

checking_balance        394
months_loan_duration      0
amount                    0
savings_balance         183
installment_rate          0
age                       0
existing_credits          0
dependents                0
employment_length_m      62
residence_history_m     130
dtype: int64

In [62]:
for c in CONTINUOUS_VARIABLES:
    df[c] = df[c].fillna(df[c].median())

#### Scaling

In [63]:
for c in CONTINUOUS_VARIABLES:
    df[c] = (df[c] - df[c].mean()) / df[c].std()

## 4. Model training
### 4.1. Cross-validation preparation

In [64]:
n_cv_groups = 10

In [65]:
k_fold = KFold(n_splits=n_cv_groups, shuffle=True, random_state=0)

In [66]:
i = 1

for _, test_subset_row_numbers in k_fold.split(df):

    test_subset_index = df.iloc[test_subset_row_numbers].index
    
    df.loc[test_subset_index, 'cv_group'] = i
    
    i += 1

In [67]:
df['cv_group'] = df['cv_group'].astype(int)

In [68]:
df['cv_group'].value_counts(dropna=False).sort_index()

1     100
2     100
3     100
4     100
5     100
6     100
7     100
8     100
9     100
10    100
Name: cv_group, dtype: int64

### 4.2. Model training

In [69]:
DUMMY_VARIABLES = dummies_df.columns.tolist()

In [70]:
PREDICTORS = DUMMY_VARIABLES + CONTINUOUS_VARIABLES

In [71]:
len(PREDICTORS)

50

In [72]:
for i in range(1, n_cv_groups + 1):
#     print('cv step {}'.format(i))
    
    train_df = df[df['cv_group'] != i]
    test_df = df[df['cv_group'] == i]
    
    train_index = train_df.index
    test_index = test_df.index
    
    X_train = train_df[PREDICTORS].values
    y_train = train_df[TARGET].values

    X_test = test_df[PREDICTORS].values
    y_test = test_df[TARGET].values
    
#     print(X_train.shape)
#     print(y_train.shape)
#     print()
#     print(X_test.shape)
#     print(y_test.shape)
#     print()
    
    lr = LogisticRegression(solver='lbfgs', class_weight='balanced', random_state=0)

    _ = lr.fit(X_train, y_train)
    
    p_train_col_name = 'p_train_{}'.format(i)
    p_test_col_name = 'p_test'
    
    df.loc[train_index, p_train_col_name] = lr.predict_proba(X_train)[:, 1]

    df.loc[test_index, p_test_col_name] = lr.predict_proba(X_test)[:, 1]

In [73]:
p_train_cols = ['p_train_{}'.format(i) for i in range(1, n_cv_groups + 1)]

In [74]:
df['p_train'] = df[p_train_cols].mean(axis=1)

### 4.3. Performance evaluation

In [75]:
p_train_pred = df['p_train'].values
p_test_pred = df['p_test'].values
y = df[TARGET].values

In [76]:
print(p_train_pred.shape)
print(p_test_pred.shape)
print(y.shape)

(1000,)
(1000,)
(1000,)


In [77]:
def log_loss(y, a):
    return - (y * np.log(a) + (1 - y) * np.log(1 - a)).mean()

In [78]:
log_loss_train = log_loss(y, p_train_pred)
log_loss_train

0.560003672175114

In [79]:
log_loss_test = log_loss(y, p_test_pred)
log_loss_test

0.6079798290884321

In [80]:
df[['p_train', 'p_test', TARGET]].head(10)

,p_train,p_test,default
index,,,
0,0.050,0.039,0
1,0.661,0.608,1
2,0.220,0.215,0
3,0.664,0.741,0
4,0.758,0.744,1
5,0.847,0.861,0
6,0.273,0.232,0
7,0.387,0.347,0
8,0.186,0.233,0


## Conclusion
Final ``log_loss`` is a bit higher on test data, which is natural.

Eventually ``p_test`` may be understood as a *probability of default* for previously unseen client.

The simplest way of ``p_test`` value usage is math rounding for yes/no decision. More complex way is a values range cut into chunks with different limit policy, however for such approach historical performance data on behaviour of similar clients is needed.